# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [3]:
df_train[['na_1', 'na_2']] = df_train[['measurement_{}'.format(i) for i in [3, 5]]].isna()
df_test[['na_1', 'na_2']] = df_test[['measurement_{}'.format(i) for i in [3, 5]]].isna()

In [4]:
# 방법 1: product_code 별로 IterativeImputer를 통한 결측처리를 하고 바로 적용합니다.

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

imp = IterativeImputer(
    estimator=LinearRegression(fit_intercept=True),
    random_state=123
)
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=x.columns)
)

df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=x.columns)
)

X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)

In [5]:
# 방법 2 : train / test를 통합하여 처리해봅니다.

from sklearn.experimental import enable_iterative_imputer# 구문을 사용하여 실험 단계인 모듈을 활성화하고, 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
# train에 등장하지 않은 수준이 있습니다, test를 포함하여 결측처리 모델을 만듭니다.
s_imp = pd.concat([
        df_train[X_imp + ['product_code']],
        df_test[X_imp + ['product_code']]
], axis=0).groupby('product_code')\
.apply(
    lambda x: IterativeImputer(estimator=LinearRegression(),random_state=123).fit(x[X_imp])
)
# train에 적용합니다.
df_train[X_imp] = df_train[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
# test에 적용합니다.
df_test[X_imp] = df_test[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
# 역시 train에 등장하지 않은 수준을 처리하기 위해 합치니다.
df_mean = pd.concat([
            df_train[['product_code'] + X_mean],
            df_test[['product_code'] + X_mean]
        ]).groupby('product_code')[X_mean].agg('mean')

df_train[X_mean] = df_train.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))
df_test[X_mean] = df_test.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))

In [6]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

In [7]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [8]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [9]:
# Test에서는 Train에 등장하지 않은 product_code가 등장합니다. 이러한 특성을 반영하는 검증법입니다.
from sklearn.model_selection import GroupKFold

# 만일 이러한 기준이 되는 범주에 범주수가 충분하여, 
# n_splits = 5, 10개 등으로 지정해도 불균형하지 않다면 임의성을 가지고 n_splits를 지정할 수 있습니다.
gcv = GroupKFold(4) # n_splits 4를 설정한 건 train의 케이스가 4개라서 입니다. 

for train_idx, valid_idx in gcv.split(df_train, groups=df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(), 
        df_train.iloc[valid_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


**일반 문제에서 얻은 힌트를 정리해봅니다.**

- log 자연 로그 함수를 적용하여 파생 변수 loading_log
- LogisticRegression X_sfs_best = ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1'] 0.5838326230092876
- LinearDiscriminantAnalysis() measurement_0 ~ 17 ?
- PCA() measurement_0 ~ 17 , n_components=7 0.581757510516433
- RandomForestClassifier(random_state=123) 'n_estimators': 15, 'max_depth': 7, 'min_samples_split': 512} 0.5745226991354744

# Kaggle형 풀이 단계

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다.

Step 3: 모델 선택 루틴을 만듭니다.

Step 4: 모델 개선 작업을 합니다.

## Step1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

In [10]:
s_hist = list()

from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

gcv = GroupKFold(4)
X_all = df_test.columns.tolist()

def eval_model(model_name, clf):
    result = cross_validate(
        clf, df_train[X_all], df_train['failure'], groups=df_train['product_code'], cv=gcv, return_train_score=True, 
        scoring='roc_auc'
    )
    
    s = 'Valid: {:.5f}±{:.5f},  V.Train: {:.5f}±{:.5f}'.format( 
            np.mean(result['test_score']), np.std(result['test_score']),
            np.mean(result['train_score']), np.std(result['train_score']),
        )
    print(s)
    s_hist.append(pd.Series({'model': model_name, 'result': s}))
    display(pd.DataFrame(s_hist).groupby('model').last())
    
def select_model(clf):
    clf.fit(df_train[X_all], df_train['failure'])
    prd = clf.predict_proba(df_test[X_all])[:, 1]
    pd.DataFrame({
        'id': df_test.index,
        'failure': prd
    }).to_csv('answer6.csv', index=None)
    return prd

## Step2: Baseline 모델을 만듭니다

In [11]:
from sklearn.linear_model import LogisticRegression

# Baseline: LR - SFS - ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
reg_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
eval_model('baseline', reg_lr)

Valid: 0.58937±0.00379,  V.Train: 0.59191±0.00146


,result
model,
baseline,"Valid: 0.58937±0.00379, V.Train: 0.59191±0.00146"


## Step3: 모델 선택 루틴을 만듭니다.

In [12]:
prd = select_model(reg_lr)
print("자가 채점: {}".format(roc_auc_score(df_ans['failure'], prd)))

자가 채점: 0.5883988309352517


## Step4: 모델 개선 작업을 합니다.

In [13]:
# lr2: LR + feature PCA(n_components = 7)
from sklearn.decomposition import PCA

ct = ColumnTransformer([
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)), ['measurement_{}'.format(i) for i in range(18)]),
    ('std', StandardScaler(), ['loading']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr2 = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
eval_model('lr2', clf_lr2)

Valid: 0.58857±0.00294,  V.Train: 0.59163±0.00103


,result
model,
baseline,"Valid: 0.58937±0.00379, V.Train: 0.59191±0.00146"
lr2,"Valid: 0.58857±0.00294, V.Train: 0.59163±0.00103"


In [14]:
# lr3: Basline + np.log (loading)
from sklearn.preprocessing import FunctionTransformer

ct = ColumnTransformer([
    ('std', StandardScaler(), ['measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('log_std', make_pipeline(FunctionTransformer(func=np.log, validate=False), StandardScaler()), ['loading']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr3 = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
eval_model('lr3', clf_lr3)

Valid: 0.58935±0.00389,  V.Train: 0.59181±0.00145


,result
model,
baseline,"Valid: 0.58937±0.00379, V.Train: 0.59191±0.00146"
lr2,"Valid: 0.58857±0.00294, V.Train: 0.59163±0.00103"
lr3,"Valid: 0.58935±0.00389, V.Train: 0.59181±0.00145"


In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# LDA
# 속성선택도 적용해봅니다: 
clf_lda = make_pipeline(
    ColumnTransformer([
        ('std', StandardScaler(), ['measurement_{}'.format(i) for i in range(18)] + ['loading']),
        #('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
        ('pt', 'passthrough', ['na_1'])
    ]), 
    LinearDiscriminantAnalysis()
)
eval_model('lda', clf_lda)

Valid: 0.58963±0.00396,  V.Train: 0.59196±0.00148


,result
model,
baseline,"Valid: 0.58937±0.00379, V.Train: 0.59191±0.00146"
lda,"Valid: 0.58963±0.00396, V.Train: 0.59196±0.00148"
lr2,"Valid: 0.58857±0.00294, V.Train: 0.59163±0.00103"
lr3,"Valid: 0.58935±0.00389, V.Train: 0.59181±0.00145"


In [16]:
# RF: RandomForestClassifier: {'max_depth': 7, 'min_samples_split': 512, 'n_estimators': 15} 튜닝
# TODO: 수업시간에 튜닝을 해봅니다.

from sklearn.ensemble import RandomForestClassifier
ct = ColumnTransformer([
    ('pt', 'passthrough', ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)])
])
clf_rf = make_pipeline(ct, RandomForestClassifier(
    n_estimators=15, max_depth=7, min_samples_split=512, random_state=123
))
eval_model('rf', clf_rf)

Valid: 0.57162±0.00305,  V.Train: 0.63545±0.00143


,result
model,
baseline,"Valid: 0.58937±0.00379, V.Train: 0.59191±0.00146"
lda,"Valid: 0.58963±0.00396, V.Train: 0.59196±0.00148"
lr2,"Valid: 0.58857±0.00294, V.Train: 0.59163±0.00103"
lr3,"Valid: 0.58935±0.00389, V.Train: 0.59181±0.00145"
rf,"Valid: 0.57162±0.00305, V.Train: 0.63545±0.00143"


In [17]:
# RF: RandomForestClassifier + LinearDiscriminantAnalysis
# TODO: LDA를 결합시키는 루틴을 만들어 보겠습니다.

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

ct = ColumnTransformer([
    # TODO
    ('pt', 'passthrough', ['loading', 'na_1', 'na_2'])# + ['measurement_{}'.format(i) for i in range(18)])
])
clf_rf2 = make_pipeline(ct, RandomForestClassifier(
    n_estimators=100, max_depth=6, min_samples_split=512, random_state=123, max_features=0.75
))
eval_model('rf2', clf_rf2)

Valid: 0.58291±0.00550,  V.Train: 0.60869±0.00074


,result
model,
baseline,"Valid: 0.58937±0.00379, V.Train: 0.59191±0.00146"
lda,"Valid: 0.58963±0.00396, V.Train: 0.59196±0.00148"
lr2,"Valid: 0.58857±0.00294, V.Train: 0.59163±0.00103"
lr3,"Valid: 0.58935±0.00389, V.Train: 0.59181±0.00145"
rf,"Valid: 0.57162±0.00305, V.Train: 0.63545±0.00143"
rf2,"Valid: 0.58291±0.00550, V.Train: 0.60869±0.00074"


In [18]:
# XGB: XGBoost
# TODO: 튜닝을 같이 해봅니다.

import xgboost as xgb
ct = ColumnTransformer([
    ('pt', 'passthrough', ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)])
])
clf_xgb = xgb.XGBClassifier()
clf_xgb = make_pipeline(ct, clf_xgb)
eval_model('xgb', clf_xgb)

Valid: 0.57835±0.00367,  V.Train: 0.68063±0.00387


,result
model,
baseline,"Valid: 0.58937±0.00379, V.Train: 0.59191±0.00146"
lda,"Valid: 0.58963±0.00396, V.Train: 0.59196±0.00148"
lr2,"Valid: 0.58857±0.00294, V.Train: 0.59163±0.00103"
lr3,"Valid: 0.58935±0.00389, V.Train: 0.59181±0.00145"
rf,"Valid: 0.57162±0.00305, V.Train: 0.63545±0.00143"
rf2,"Valid: 0.58291±0.00550, V.Train: 0.60869±0.00074"
xgb,"Valid: 0.57835±0.00367, V.Train: 0.68063±0.00387"


In [19]:
# Voting을 합니다.
from sklearn.ensemble import VotingClassifier
# TODO 
# clf_vt = VotingClassifier
# eval_model('vt', clf_vt)